In [1]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd


In [7]:
# get events from n days ago
look_back_days = 45

days_ago_dte = dt.now() - timedelta(days=look_back_days)
print (days_ago_dte, type(days_ago_dte)) #gives us a time object
days_ago = days_ago_dte.strftime('%Y-%m-%d')
print (days_ago, type(days_ago)) #converts the time object to string so it can be based into the request

# you can limit the number of records that return from the API
# set to 0 for no limit
limit = 100

api_url = 'https://api.acleddata.com/acled/read?terms=accept&event_date={}&event_date_where=%3E=&limit={}'.format(days_ago, limit)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

2020-08-23 11:00:28.159076 <class 'datetime.datetime'>
2020-08-23 <class 'str'>
https://api.acleddata.com/acled/read?terms=accept&event_date=2020-08-23&event_date_where=%3E=&limit=100 <class 'str'>


In [8]:
response = requests.get(api_url) # Reality I spent 45 minutes figuring out why my search kept failing 
                                # and learned I had an older user guide that didn't include terms=accept 
data = response.json()
data.keys()

dict_keys(['status', 'success', 'last_update', 'count', 'messages', 'data', 'filename'])

In [9]:
data['status'], data['success'], data['count'] # see the refrenet for API status codes

(200, True, 100)

In [10]:
data_new = data['data'] #getting just the data 
rows = len(data_new)
print ('ACLED API returned {} rows'.format(rows))

ACLED API returned 100 rows


In [11]:
#convert it into an pandas data frame
acled_data = pd.DataFrame.from_dict(data_new)
acled_data

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7242688,31,AZE16293,16293,2020-10-03,2020,1,Battles,Armed clash,Military Forces of Azerbaijan (2003-),...,Artsakh-Azerbaijan LoC (Unspecified),39.8910,47.1018,3,Tert,Regional,"On 3 October 2020, the ministry of defense of ...",51,1602027397,AZE
1,7242682,31,AZE16309,16309,2020-10-03,2020,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Azerbaijan (2003-),...,Stepanakert,39.8246,46.7535,1,Tert,Regional,"On 3 October 2020, military forces of Azerbaij...",0,1602027397,AZE
2,7242645,31,AZE16301,16301,2020-10-03,2020,1,Battles,Government regains territory,Military Forces of Azerbaijan (2003-),...,Mataghis,40.3278,46.7342,1,APA,National,"On 3 October 2020, the president of Azerbaijan...",0,1602027397,AZE
3,7242644,31,AZE16288,16288,2020-10-03,2020,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Artsakh (2007-),...,Quzanli,40.1568,47.1619,1,APA,National,"On 3 October 2020, the Ministry of Defense of ...",0,1602027397,AZE
4,7242643,31,AZE16291,16291,2020-10-03,2020,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Artsakh (2007-),...,Qiyamaddinli,39.9743,47.1124,1,APA,National,"On 3 October 2020, the Ministry of Defense of ...",0,1602027397,AZE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7241473,4,AFG48128,48128,2020-10-03,2020,1,Violence against civilians,Attack,Unidentified Armed Group (Afghanistan),...,Kabul,34.5167,69.1833,1,Pajhwok Afghan News,National,"On 3 October 2020, Maidan Shar city mayor was ...",0,1602014401,AFG
96,7241455,268,GEO2303,2303,2020-10-03,2020,1,Riots,Mob violence,Rioters (Georgia),...,Bolnisi,41.4503,44.5450,1,Netgazeti.ge,National,"On 3 October 2020, police arrived in Bolnisi t...",0,1602014401,GEO
97,7241180,268,GEO2304,2304,2020-10-03,2020,1,Protests,Peaceful protest,Protesters (Georgia),...,Marneuli,41.4741,44.7896,1,Formula TV,National,"On 3 October 2020, United National Movement or...",0,1602014400,GEO
98,7241166,4,AFG48144,48144,2020-10-03,2020,2,Battles,Armed clash,Taliban,...,Sultan Khel,33.8816,68.6317,1,Bakhtar News Agency,National,"Around 3 October 2020 (as reported), 4 Taliban...",1,1602014400,AFG


### Does this comport with what is the in the documentation? 